In [1]:
import sys
sys.path.insert(0,'/home/pgaemers/software/epix')

In [2]:
import time
import pandas as pd
import straxen
import epix
from epix.simulator.fast_simulator import StraxSimulator

import numpy as np
print(f'epix version : {epix.__version__}')

Using nestpy version 1.3.0
epix version : 0.0.6


In [3]:
def monitor_time(prev_time, task):
    t = time.time()
    print(f'It took {(t - prev_time):.4f} sec to {task}')
    return t

In [4]:
# MC output file as an input for epix/fast_simulator

raw_data_dir='/dali/lgrandi/xenonnt/simulations/testing_epix_wfsim/'
raw_data_filename='tpc_and_nveto_cryoneutrons_200.root'
st = straxen.contexts.xenonnt_simulation(output_folder='/dali/lgrandi/pgaemers/strax_data_fast_sim')
run_id = '1'
st.register(StraxSimulator)
st.set_config=(dict(nchunk=1, event_rate=5, chunk_size=500,))

In [5]:
# %rm -r './strax_data'

nt_config=straxen.get_resource('/home/pgaemers/software/private_nt_aux_files/\
sim_files/fax_config_nt_low_field.json', fmt='json')

# epix config

epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':1000,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.005,
           'tag_cluster_by':'time',
           'max_delay':1e-7,
           'source_rate':-1}

# configuration files; to be moved into the plugin

configuration_files={'s1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_pattern_map':'XENONnT_s1_xyz_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     's2_pattern_map':'XENONnT_s2_xy_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     'nv_pmt_qe':'nveto_pmt_qe.json'}

st.config.update(dict(fax_config=nt_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      xy_resolution=0.5,
                      z_resolution=0.5,
                      configuration_files=configuration_files),
                nv_spe_resolution=0.35,
                nv_spe_res_threshold=0.5,
                nv_max_coin_time_ns=50000,
                dpe_fraction=1.)

In [7]:
#st.config
#st.make(run_id, 'events_full_simmed')

sim=StraxSimulator()
sim.config=st.config
sim.setup()

t_start = time.time()
output_nveto_hits=sim.compute()
_=monitor_time(t_start, 'run epix and compute.')

It took 1.3660 sec to run epix and compute.


In [9]:
output_nveto_hits['eventid']

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [12],
 [14],
 [15],
 [16],
 [18],
 [19],
 [20],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [29],
 [30],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [47],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [87],
 [88],
 [89],
 [90],
 [92],
 [93],
 [94],
 [95],
 [97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104],
 [105],
 [106],
 [107],
 [109],
 [110],
 [111],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118],
 [119],
 [120],
 [121],
 [122],
 [123],
 [124],
 [125],
 [126],
 [127],
 [128],
 [129],
 [130],
 [131],
 [133],
 [136],
 [137],
 [138],
 [141],
 [142],
 [143],
 [144],
 [145],
 [146],
 [147],
 [148],
 [149],
 [150],
 [151],
 [152],
 [153],
 [154],
 [155],
 [156],
 [157],
 [158],

In [ ]:
# Simulator output, nVeto hits: each event has a list of pairs [timestamp, pmt_channel]; each pair corresponds to a generated photoelectron

# output_nveto_df=pd.DataFrame(output_nveto_hits)
# output_nveto_df[output_nveto_df.hits.apply(lambda x: len(x) > 0)].head()

In [ ]:
import json

In [10]:
def get_nv_pmt_qe(pmt_json_dict, pmt_ch, photon_ev):
    wvl = (1e9 * (scp.constants.c * scp.constants.h) / (photon_ev * 1.60218e-19))

    nv_pmt_qe_wavelength = np.array(pmt_json_dict['nv_pmt_qe_wavelength'])
    nv_pmt_qe = pmt_json_dict['nv_pmt_qe']

    wvl_index = np.abs(nv_pmt_qe_wavelength - wvl).argmin()

    return nv_pmt_qe[str(pmt_ch)][wvl_index]
class Resource():
    def __init__(self,) -> None:
        self._load_resource()

    def _load_resource(self,):
        '''Loads needed configs to call wfsim. We need s1/s2 light yield maps,
        spe distibutions and corrections maps'''
        self.nveto_pmt_qe = json.loads(straxen.get_resource('nveto_pmt_qe.json'))

In [89]:
ttree, _ = epix.io._get_ttree(raw_data_dir,raw_data_filename)
resource = Resource()
pmt_nv_json_dict = resource.nveto_pmt_qe
SPE_Resolution=0.35, 
SPE_ResThreshold=0.5,
max_coin_time_ns=500.0,
batch_size=10000
nveto_dtype=[('event_id',np.int),('channel',np.int),('time',np.float),('endtime',np.float)]

In [38]:
import uproot
import collections
import scipy as scp

In [91]:
hits_dict = {'event_id': [], 'time': [],'channel': []}
num_hits=0
for events_iterator in uproot.iterate(ttree,
                                  ['eventid', 'pmthitID', 'pmthitTime', 'pmthitEnergy'],
                                  step_size=batch_size,
                                  ioutputtype=collections.namedtuple):
    for eventid_evt, pmthitID_evt, pmthitTime_evt, \
        pmthitEnergy_evt in zip(getattr(events_iterator, 'eventid'),
                                getattr(events_iterator, 'pmthitID'),
                                getattr(events_iterator, 'pmthitTime'),
                                getattr(events_iterator, 'pmthitEnergy')):
        hit_list = []
        hit_coincidence = 0

        pmt_in_coincidence = []
        pe_per_pmt = []

        for _time, _id, _energy in zip(pmthitTime_evt, pmthitID_evt, pmthitEnergy_evt):
            if _id >= 2000 and _id < 2120:

                qe = 1e-2 * get_nv_pmt_qe(pmt_nv_json_dict, _id, _energy)

                pe = np.random.binomial(1, qe, 1)[0]
                if pe < 0.5:
                    continue

                pe_res = np.random.normal(1.0, SPE_Resolution, 1)
                if pe_res >= SPE_ResThreshold:
                    hit_list.append([_time * 1e9, _id])

        hit_array = np.array(hit_list)
        pmt_coincidence_dict = None
        hit_array_coincidence = np.array([])

        if hit_array.shape[0] > 0:
            t0 = hit_array[:, 0].min()
            tf = t0 + max_coin_time_ns
            hit_array_coincidence = hit_array[hit_array[:, 0] < tf]
        if len(hit_array_coincidence)>0:
            hits_dict['event_id'].append([eventid_evt]*len(hit_array_coincidence))
            hits_dict['time'].append(hit_array_coincidence[:,0])
            hits_dict['channel'].append(hit_array_coincidence[:,1])
            num_hits+=len(hit_array_coincidence)
            
result = np.zeros(num_hits,dtype=nveto_dtype)
result['event_id']=list(itertools.chain.from_iterable(hits_dict['event_id']))
result['channel']=list(itertools.chain.from_iterable(hits_dict['channel']))
result['time']=list(itertools.chain.from_iterable(hits_dict['time']))
result['endtime']=result['time']+1

In [95]:
result['endtime']

array([6.58379680e+00, 6.58584327e+00, 7.80365979e+00, ...,
       4.04045547e+05, 4.04065525e+05, 4.04173757e+05])

In [73]:
import itertools

In [82]:
hits_dict['hits']:,1]

TypeError: list indices must be integers or slices, not tuple

In [76]:
list(itertools.chain.from_iterable(hits_dict['hits']))

[array([52687.09461307,  2030.        ]),
 array([52612.19969083,  2039.        ]),
 array([52601.40837142,  2039.        ]),
 array([52583.71148739,  2041.        ]),
 array([52682.5479445,  2106.       ]),
 array([52753.38007402,  2030.        ]),
 array([52601.41130313,  2083.        ]),
 array([52578.97488451,  2044.        ]),
 array([52626.29271695,  2082.        ]),
 array([53000.7048882,  2114.       ]),
 array([52582.10454974,  2040.        ]),
 array([52618.27754513,  2097.        ]),
 array([52575.74355907,  2026.        ]),
 array([52569.60598662,  2025.        ]),
 array([52651.26395641,  2083.        ]),
 array([52587.18526893,  2032.        ]),
 array([52647.64683953,  2082.        ]),
 array([52601.96286419,  2080.        ]),
 array([52801.0557736,  2063.       ]),
 array([52570.57577878,  2042.        ]),
 array([52569.54406957,  2038.        ]),
 array([52583.94319902,  2108.        ]),
 array([52614.18854447,  2010.        ]),
 array([52578.84673369,  2020.        ])

In [61]:
import itertools

list2d = [[1,2,3], [4,5,6], [7], [8,9]]
merged = list(itertools.chain(*list2d))

30480

In [64]:
c=dict(chunk_size=5,nchunk=1,event_rate=5)

In [66]:
c['total_time'] = c['chunk_size'] * c['nchunk']
n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
time = uniform_times * int(1e9)

In [67]:
time

array([1.0e+08, 3.0e+08, 5.0e+08, 7.0e+08, 9.0e+08, 1.1e+09, 1.3e+09,
       1.5e+09, 1.7e+09, 1.9e+09, 2.1e+09, 2.3e+09, 2.5e+09, 2.7e+09,
       2.9e+09, 3.1e+09, 3.3e+09, 3.5e+09, 3.7e+09, 3.9e+09, 4.1e+09,
       4.3e+09, 4.5e+09, 4.7e+09, 4.9e+09])